# Libraries 📚

In [1]:
import requests
import pandas as pd
import json 

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

# Getting data with API 👨‍💻

## Location ❓🌍

In [ ]:
all_location_dic = []

for city in cities:
    url = "https://nominatim.openstreetmap.org/search"
    payload = {
    "city" : {city}, "country" : "France",
    "format": "json" 
    }
    location = requests.get(url, params=payload).json()

    keys_to_extract = ["name", "lat", "lon"]
    location_dic = {key : location[0][key] for key in keys_to_extract}

    all_location_dic.append(location_dic)

all_location_df = pd.DataFrame(all_location_dic)  # converting to data frame
all_location_df.insert(0, 'ID', range(1, len(all_location_df) + 1))  # creating ID column
all_location_df

In [8]:
all_location_df = pd.read_csv("gps.csv")

In [11]:
all_location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      35 non-null     int64  
 1   name    35 non-null     object 
 2   lat     35 non-null     float64
 3   lon     35 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.2+ KB


In [12]:
all_location_df['lat'] = pd.to_numeric(all_location_df['lat'], errors='coerce')
all_location_df['lon'] = pd.to_numeric(all_location_df['lon'], errors='coerce')

fig = px.scatter_mapbox(all_location_df, lat='lat', lon='lon', hover_name='name', 
                        zoom=4, height=700, color_discrete_sequence=['red'])

fig.update_layout(mapbox_style='carto-positron')
fig.show()

## What's the weather 🌞

In [13]:

from Api_key import api_key


cols= ['feelslike', 'pop', 'rain','weather_id','main_weather']

weather_df = pd.DataFrame(data=None, columns=cols, dtype=None, copy=None)


for index, row in all_location_df.iterrows():
    lat = row['lat']
    lon = row['lon']

    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=hourly,minutely,alerts&units=metric&appid={api_key}"

    w = requests.get(url)
    weather = w.json()


    for day in weather['daily']:
        newRow = {'feelslike': day['feels_like']['day'],
                'pop': day['pop'],
                'rain':  day.get('rain', 0),
                'weather_id': day['weather'][0]['id'],
                'main_weather': day['weather'][0]['main']
                }
        
    
        weather_df = pd.concat([weather_df, pd.DataFrame([newRow])], ignore_index=True)



C:\Users\Malika\AppData\Local\Temp\ipykernel_7512\1242914936.py:28: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [14]:
# add column name of cities         
city_names = [city for city in all_location_df['name'] for _ in range(8)]
weather_df['City']= city_names

In [15]:
weather_df.shape

(280, 6)

In [16]:
weather_df.nunique()

feelslike       254
pop              48
rain            160
weather_id        8
main_weather      3
City             35
dtype: int64

In [17]:
city_names = [city for city in all_location_df['ID'] for _ in range(8)]
weather_df['ID']= city_names

In [37]:
weather_df.head()

,feelslike,pop,rain,weather_id,main_weather,City,ID,city_score
0,14.85,0,0,804,Clouds,Mont Saint-Michel,1,202.275000
1,18.67,0.34,0,800,Clear,Mont Saint-Michel,1,252.631866
2,19.80,1,2.38,500,Rain,Mont Saint-Michel,1,59.285799
3,17.67,1,6.15,501,Rain,Mont Saint-Michel,1,40.491014
4,11.71,1,13.89,501,Rain,Mont Saint-Michel,1,24.280917


In [19]:
weather_df.nunique()

feelslike       254
pop              48
rain            160
weather_id        8
main_weather      3
City             35
ID               35
dtype: int64

In [20]:
def calculate_city_score(row):
    score = 0
    score += row['feelslike']*1.5

    score += (1 / (row['rain'] + 1)) * 100
    score += (1 / (row['pop'] + 1)) * 100
    
    if row['main_weather'] == 'Clear':
        score += 50
    elif row['main_weather'] == 'Rain':
        score -= 50
    elif row['main_weather'] == 'Clouds':
        score -= 20
    elif row['main_weather'] == 'Snow':
        score -= 60
    
    return score

weather_df['city_score'] = weather_df.apply(calculate_city_score, axis=1)


In [ ]:
ranked_cities = weather_df.groupby('ID')['city_score'].mean().sort_values(ascending=False).reset_index()
ranked_cities.head(10)

In [30]:
top_5_cities = ranked_cities.iloc[:5,:]
top_5_cities

,ID,city_score
0,28,165.221614
1,29,155.487763
2,31,144.634861
3,3,138.342392
4,8,133.274405


In [34]:
ranking_by_w = pd.merge(ranked_cities, all_location_df, on="ID", how="inner")
ranking_by_w.shape

(35, 5)

### Top 5 🏆

In [38]:
top_cities = ranking_by_w.loc[:4]
top_cities

,ID,city_score,name,lat,lon
0,28,165.221614,Collioure,42.525050,3.083155
1,29,155.487763,Carcassonne,43.213036,2.349107
2,31,144.634861,Toulouse,43.604462,1.444247
3,3,138.342392,Bayeux,49.276462,-0.702474
4,8,133.274405,Lille,50.636565,3.063528


In [40]:
fig = px.scatter_mapbox(ranking_by_w, 
                        lat="lat", 
                        lon="lon", 
                        hover_name="name", 
                        zoom=4,
                        color="city_score",
                        color_continuous_scale=px.colors.sequential.Rainbow,
                        size='city_score',
                        opacity=0.7,
                        mapbox_style="carto-positron",
                        title="City Scores"
                       )
fig.update_layout(width=800, height=600)

fig.show()

## Exporting data 💾

In [39]:
# DF all cities (city_score, name, lat, lon) sorted by city_score
ranking_by_w.to_csv('ranking_by_weather_score_v2.csv', index= False)

# DF all cities and their Lat & Lon
all_location_df.to_csv('gps_v2.csv', index= False)


# Top five 
top_cities.to_csv('top_5_v2.csv', index=False)

#citis_weather
weather_df.to_csv('cities_weather_v2.csv', index= False)


In [44]:
top_by_w = weather_df.iloc[:5,:]
top_by_w

,feelslike,pop,rain,weather_id,main_weather,City,ID,city_score
0,14.85,0,0,804,Clouds,Mont Saint-Michel,1,202.275000
1,18.67,0.34,0,800,Clear,Mont Saint-Michel,1,252.631866
2,19.80,1,2.38,500,Rain,Mont Saint-Michel,1,59.285799
3,17.67,1,6.15,501,Rain,Mont Saint-Michel,1,40.491014
4,11.71,1,13.89,501,Rain,Mont Saint-Michel,1,24.280917


In [46]:
ranking_by_w

,ID,city_score,name,lat,lon
0,28,165.221614,Collioure,42.525050,3.083155
1,29,155.487763,Carcassonne,43.213036,2.349107
2,31,144.634861,Toulouse,43.604462,1.444247
3,3,138.342392,Bayeux,49.276462,-0.702474
4,8,133.274405,Lille,50.636565,3.063528
5,5,133.054032,Rouen,49.440459,1.093966
6,7,131.641212,Amiens,49.894171,2.295695
7,17,130.058913,Lyon,45.757814,4.832011
8,6,127.452136,Paris,48.853495,2.348391
9,32,121.894744,Montauban,44.017584,1.354999
